# PyTorch Experiment Tracking #

We've trained and created a couple of models already from basic linear neural networks to convolutional networks such as TinyGG. So far, we've been just keeping tracking of them through Python dictionaries and also by comparing them on the metric print outs during training. 

Now, that's all well and good but what if we want to run a couple of different models at once?

**Experiment Tracking** is exatly how we do that. 

Experiment tracking is an important aspect and is one of the integral parts of machine learning. In this chapter, we're going to answer just one question - **How do you track your machine learning experiments?**

But before we do that, let's ask the simple question first. 

**What is Experiment Tracking?**

You might've noticed already but we're doing a lot of experiments when it comes machine learning simply because there are a lot of different variables involved that you can change. From the hyperparameters, models, data, and many more. 

Because of this, we need something to properly track the results of these many different experiments. That is why experiment tracking exists and why it is so imporant because it enables us to check and compare our findings. We get to know what works and what doesn't fast. 

This saves us time and effort which can be better used on other areas. 

**Why Track Experiments?**

Before, we were running only a few models and we weren't doing a lot of changing with each model that we make so our previous process of using print outs and sending the results to a simple dictionary was a good enough solution. BUT we always reitirated the motto 'experiment experiment experiment' so eventually, as you get better with dealing with models, you're going to get curious. 

With that, you'll eventually be dealing with many different variations of potential solutions to your problems so you'll want something to record these results.

![Display](images/08-experiment-tracking-can-get-out-of-hand.png)

Let's take a look at a few ways you can track the results of your machine learning experiments

**Result Tracking Solutions**

* *Python Dict, CSV files, Print outs* - **PRO:** Easy to setup, runs in pure python | **CON:** Hard to keep track with large number of experiments, does not scale conveniently| **COST:** FREE
* *TensorBoard* - **PRO:** Extensions built directly into PyTorch, widely recognized and used, easily scales with experiments| **CON:** Not so intuitive and friendly user-experience compared to other options| **COST:** FREE
* *Weights & Biases Experiment Tracking* - **PRO:** Better user experience, can share experiment to the public, can track a wide range of items | **CON:** Requires reference materials outside of PyTorch | **COST:** FREE FOR PERSONAL USE
* *MLFlow* - **PRO:**  Open-source, flexible and many integrations | **CON:** Can be hard to setup | **COST:** FREE

![Display](images/08-different-places-to-track-experiments.png)

**Chapter Coverage**

Because the section covers experiment tracking, it goes without saying that we're going to do a lot of different modelling experiments with  varying levels of data, model size, and training time. Keeping with consistency, we're still trying to improve on FoodVision Mini

While there are many options to choose for experiment tracking, this chapter will only focus with TensorBoard because of it's tight integration with PyTorch and widespread use. But overall, the principles stay the same for all other platforms for experiment tracking.

1. **Getting Setup** - We'll download/transfer the same files that we used earlier to make data setup and training easier.
2. **Get Data** - Utilize the same data of pizza, steak, and sushi for image classification to improve FoodVision Mini Model's results.
3. **Create Datasets & DataLoaders** - Refer to *data_setup.py* for the function that can make the Dataset and DataLoader from the train and test directories of pizza, steak, sushi.
4. **Get & Customize Pretrained Model** - Pick a pretrained model from PyTorch's *torchvision.models* library and then customize that pretrained model to fit our own needs.
5. **Train Model & Track Results** - We will start to train the model and see how to track the results of a single model with TensorBoard.
6. **View Model Results In TensorBoard** - The way we viewed loss curves before was through a helper function. This time, we'll be using TensorBoard's own way of visualizing the loss curve.
7. **Creating Helper Functions to Track Experiments** - In addition to this, we will also create a helper function for us to save our model's experimenting results.
8. **Developing Series of Modelling Experiments** - Previously, we did experiments one by one. We set up one model and trained it. This time, we'll create some code to run multiple exeriments at once with different modes, amounts of data, and training time.
9. **Viewing Modelling Experiments in TensorBoard** - Overall, at this stage of the chapter there should be around eight modelling experiments running consecutively. We'll take a look at all the results for each model in TensorBoard.
10. **Loading Best Model & Creating Predictions** - Once we've picked out the model that has the best results, we'll use it practically and visualize the these predictions to see it in action. Visualize! Visualize! Visualize!

**Getting Setup**

Before we get started, we'll need the all the modules. Again, we'll be using the *data_setup.py* and *engine.py* from the modularity chapter. We will also import *torchinfo* for visualizing the sumarries of the model.

In [1]:
%load_ext autoreload
%autoreload 2

In [59]:
import torch
import torchvision

import os
import zipfile
import requests
import matplotlib.pyplot as plt
import random

from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter
from scripts import data_setup, utils, predict
from scripts.engine import train_step, test_step
from scripts import engine

from pathlib import Path
from typing import List, Dict, Tuple
from tqdm.auto import tqdm
from datetime import datetime

from mlxtend.plotting import plot_confusion_matrix
from torchmetrics import ConfusionMatrix
from PIL import Image

In [60]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

**Create Helper Function To Generate Seeds**

We've always been setting seeds manually before so it was a bit tedious. Let's just turn it into a function so that everything is a lot easier. Create a function named *set_seeds()* that we can just call anytime we need to create a seed. Set the default seed to 42.

In [61]:
def set_seeds(seed:int = 42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

**Get Data**

Since we're trying to improve the results from FoodVision Mini, we'll still use the same dataset as before - pizza, steak, sushi. We've already seen how powerful pretrained models are when used with our custom dataset but that was just one of many configurations that we can use. There are so many different pretrained models out there that we can try and many varying optimizers and hyperparameters to change. 

That's why we're still working with the same dataset. With that said, let's download the data again for this chapter. We'll still be working with a lot of similiar code. 

In [62]:
def download_data(
    src: str,
    dest: str,
    remove_source: bool = True) -> Path:
    
    data_path = Path('data')
    image_path = data_path/dest

    if image_path.is_dir():
        print(f"Directory {image_path} Already Exists! Cancelling ...")
    else:
        print(f"Directory {image_path} Not Found! Creating ...")
        image_path.mkdir(parents=True, exist_ok=False)

    target_file = Path(src).name
    print(target_file)
    
    with open(data_path/target_file, "wb") as f:
        request = requests.get(src)
        print(f"[INFO] Downloading {target_file} from {src} ...")
        f.write(request.content)

    with zipfile.ZipFile(data_path/target_file, "r") as zf: 
        print(f"[INFO] Unzipping {target_file} ...")
        zf.extractall(image_path)

    if remove_source:
        os.remove(data_path/target_file)

    return image_path 

In [63]:
dataset_path = download_data(src='https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip', dest='pizza_steak_sushi')

Directory data\pizza_steak_sushi Already Exists! Cancelling ...
pizza_steak_sushi_20_percent.zip
[INFO] Downloading pizza_steak_sushi_20_percent.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip ...
[INFO] Unzipping pizza_steak_sushi_20_percent.zip ...


**Creating Datasets & DataLoaders**

Time to turn our data to Datasets and DataLoaders. Use the *create_dataloaders()* function from *data_setup.py*. At the same time, manually create transforms with *torchvision.transforms*. After this, create an automated transform from a pretrained *torchvision* model.

Remember the benefits of each - for manually creating transforms, you get access to unlimited flexibility because you can add in extra data augmentation transforms but at the cost of having to suffer from potential performance degredation because the transforms don't exactly match the pretrained model's transform. 

Typically, the way to avoid this performance degredation is by normalizing the images in ImageNet format. Why just ImageNet format? It's because that is the dataset where all pretrained *torchvision.models* are trained on. These normalize values are provided in documentation of the models in PyTorch.

**Manual Transform Creation**

In [64]:
manual_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std=[0.229,0.224,0.225])
])

manual_transforms

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [65]:
train_dir = dataset_path/'train'
test_dir = dataset_path/'test'

train_dataloader, test_dataloader, class_list = data_setup.create_dataloaders(train_dir=train_dir, 
                                                                              test_dir=test_dir, 
                                                                              transforms=manual_transforms,
                                                                              batch_size=32,
                                                                              num_workers=0
                                                                             )

train_dataloader, test_dataloader, class_list

(<torch.utils.data.dataloader.DataLoader at 0x1bd1af8b3e0>,
 ['pizza', 'steak', 'sushi'])

**Auto Transform Creation**

In [66]:
weights = models.EfficientNet_B0_Weights.DEFAULT
model = models.efficientnet_b0(weights=weights)

In [67]:
auto_weights = weights.transforms()
auto_weights

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

**Getting Pretrained Layers, Freezing Base Layers, Changing Classifier Head**

Before we start runinng multiple modelling experiments at once, we'll try just running one. We have our data ready so let's start working with our pretrained models. Choose one and then freeze the base layers so that we can get a feature extractor model.

Remember that feature extractor models are different from fine-tuned models. The difference is that feature extraction does not change much in the base layers and only changes the classifier heads. While fine-tuned involves changing certain base layers or all of it. 

In this case, we're only going for a feature extractor model. Just like before, freeze the base layers and change the classifier head. 

In [68]:
summary(model=model,
        input_size=(32,3,224,224), # ensure that this is input_size and not input_shape
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 1000]           --                   True
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   True
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   True
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   864                  True
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   64                   True
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 16, 112

In [69]:
for param in model.features.parameters():
    param.requires_grad = False


model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(1280, len(class_list), bias=True)
)

In [70]:
summary(model=model,
        input_size=(32,3,224,224), # ensure that this is input_size and not input_shape
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Sequential (features)                                      [32, 3, 224, 224]    [32, 1280, 7, 7]     --                   False
│    └─Conv2dNormActivation (0)                              [32, 3, 224, 224]    [32, 32, 112, 112]   --                   False
│    │    └─Conv2d (0)                                       [32, 3, 224, 224]    [32, 32, 112, 112]   (864)                False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]   [32, 32, 112, 112]   (64)                 False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]   [32, 32, 112, 112]   --                   --
│    └─Sequential (1)                                        [32, 32, 112, 112]   [32, 

**Train & Track Results**

We've got the data and model ready. Time to get started on training. Let's assign the loss functions and optimizer. Same as usual, choose the *torch.nn.CrossEntropyLoss* for the loss function and for optimizer, experiment around with *torch.optim.SGD* or *torch.optim.Adam*. Assign the learning rate to 0.001

In [71]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

**Adjust Train() Function To Track Results With SummaryWriter()**

Here starts the new concepts and ideas. We've been tracking the results of our *train()* function with multiple Python dictionaries - one for each model. It goes without saying that this process doesn't scale easily when you're dealing with more than one experiment.

Introducing PyTorch's *torch.utils.tensorboard.SummaryWriter()* class that saves various parts of the model's training progress to file. 

*SummaryWriter()* saves information about the model to a file set by the *log_dir* parameter. The default location for *log_dir* is *run/CURRENT_DATETIME_HOSTNAME* of which *HOSTNAME* is the name of your computer and the associated datetime when the model was trained.

You can always change where the experiment results are sent. The filename can be changed to anything you want. The outputs of the *SummaryWriter()* class are all saved in *TensorBoarad* format. 

TensorBoard is a part of the TensorFlow deep learning library. TensorFlow might be something you've heard before because it is one of the direct competitors of PyTorch. Luckily, we can always work with PyTorch and a part of TensorFlow - TensorBoard. 

Let's start working with *SummaryWriter()*

In [72]:
writer = SummaryWriter()

Because we'll be changing up the method we use to track the results, naturally, our previous function for training models in *engine.py* wouldn't work without some adjustments to the code. Now, we could make an entirely new train function but we'll opt to changing certain aspects only to save time and just be more efficient. 

Now, what are we going to change? We will be adding the feature for the *train()* function to log the model's training and test loss + accuracy values much like as to what we were doing before but this time with TensorBoard.

We'll do this with *writers.add_scalars(main_tag, tag_scalar_dict)*. What does these two parameters do exactly?

* *main_tag* (string) - this is the name of the scalar that is being tracked such as *'accuracy'* or *'loss'*
* *tag_scalar_dict* (dict) - this is the dictionary of values being tracked such as *{'train_loss: 0.3454'}*

Remember that loss and accuracy are scalars, that's why the function is called *add_scalars()*. 

Once we're done with tracking values that we want, we call *writer.close()* so that the *writer* class would stop looking for values to track. We don't need to remake everything. We can just simply use the same *train_step* and *test_step* from *engine.py* while the only function that we need to work with is *train()*.

Take note that you are able to track information on the model almost anywhere in code but it is most often that experiments are tracked while a model is training. The *torch.utils.tensorboard.SummaryWriter()* class does have additional methods to track things in the model/data. Example of this is *add_graph()* of which tracks the computation graph of the model. There are many more to check out, visit the *SummaryWriter* documentation to check.

In [73]:
def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    epochs: int,
    device: torch.device
) -> Dict [str, List]:

    # Creating empty results dictionary
    results = {
        "train_loss": [],
        "test_loss": [],
        "train_acc": [],
        "test_acc": []
    }

    # Looping through train and test steps for number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device
        )

        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device
        )

        # Printing out the outputs
        print(
            f"Epoch: {epoch+1} | "
            f"Train Loss {train_loss:.4f} | "
            f"Train Accuracy {train_acc:.4f} | "
            f"Test Loss {test_loss:.4f} | "
            f"Test Accuracy {test_acc:.4f} | "
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)


        ### NEW !! - Experiment Tracking 

        # Adding loss results to SummaryWriter
        writer.add_scalars(
            main_tag='Loss',
            tag_scalar_dict={"train_loss": train_loss,
                             "test_loss": test_loss},
            global_step=epoch
        )

        # Adding accuracy results to SummaryWriter
        writer.add_scalars(
            main_tag='Accuracy',
            tag_scalar_dict={"train_acc": train_acc,
                             "test_acc": test_acc},

            global_step=epoch
        )

        # Track PyTorch Model Architecture
        writer.add_graph(model=model,
                          input_to_model=torch.randn(32,3,256,256).to(device))

    writer.close()
    return results

In [74]:
set_seeds(42)
results = train(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=10,
    device=device
)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Train Loss 0.9918 | Train Accuracy 0.5125 | Test Loss 0.7034 | Test Accuracy 0.8419 | 
Epoch: 2 | Train Loss 0.6824 | Train Accuracy 0.8292 | Test Loss 0.5289 | Test Accuracy 0.9032 | 
Epoch: 3 | Train Loss 0.5313 | Train Accuracy 0.8938 | Test Loss 0.4380 | Test Accuracy 0.9121 | 
Epoch: 4 | Train Loss 0.4628 | Train Accuracy 0.8750 | Test Loss 0.3827 | Test Accuracy 0.9211 | 
Epoch: 5 | Train Loss 0.4011 | Train Accuracy 0.9083 | Test Loss 0.3437 | Test Accuracy 0.9255 | 
Epoch: 6 | Train Loss 0.3671 | Train Accuracy 0.9000 | Test Loss 0.3124 | Test Accuracy 0.9255 | 
Epoch: 7 | Train Loss 0.3253 | Train Accuracy 0.9167 | Test Loss 0.2999 | Test Accuracy 0.9300 | 
Epoch: 8 | Train Loss 0.2837 | Train Accuracy 0.9458 | Test Loss 0.2802 | Test Accuracy 0.9344 | 
Epoch: 9 | Train Loss 0.2937 | Train Accuracy 0.9271 | Test Loss 0.2651 | Test Accuracy 0.9344 | 
Epoch: 10 | Train Loss 0.2666 | Train Accuracy 0.9313 | Test Loss 0.2583 | Test Accuracy 0.9344 | 


Since the *train()* function is now updated to use the *SummaryWriter()* instance in tracking the model's results. Let's try running it for 5 epochs and see what happens.

In [75]:
weights = models.ResNet18_Weights.DEFAULT
resnet = models.resnet18(weights=weights)

In [76]:
for param in resnet.parameters():
    param.requires_grad = False

for param in resnet.fc.parameters():
    param.requires_grad = True

resnet.fc = nn.Linear(in_features=512, out_features=len(class_list), bias=True)

resnet.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [77]:
summary(model=resnet,
        input_size=(32,3,224,224), # ensure that this is input_size and not input_shape
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 3]              --                   Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 64, 56, 56]     --                   False
│    └─BasicBlock (0)                    [32, 64, 56, 56]     [32, 64, 56, 56]     --                   False
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     (36,864)             False
│    │    

In [80]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)

In [81]:
results = train(
    model=resnet,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=10,
    device=device
)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 1 | Train Loss 1.0156 | Train Accuracy 0.4729 | Test Loss 0.7787 | Test Accuracy 0.7077 | 
Epoch: 2 | Train Loss 0.6995 | Train Accuracy 0.7646 | Test Loss 0.5414 | Test Accuracy 0.8943 | 


KeyboardInterrupt: 

Everything might seem the same. The records are similiar but there is a key difference in that behind the scenes, the *writer* instance has created a *runs/* directory of which the model's results are stored.  

An example of the save location would look like: 

runs/Jun24_03-35-30_josh

Remember that the default format is runs/CURRENT_DATETIME_HOSTNAME. 

Now, going back to our previos method of tracking results in a dictionary. Let's take a look at that.

In [ ]:
results

This looks very inconvenient. While there surely is a way that we can make this dictionary look more appealing but luckily, we don't have to worry much about that because we already have utilized TensorBoard. But, what does that actually look like?

**View Model Results In TensorBoard**

The *SummarWriter* class stored the model's results in a directory called 'runs/' in TensorBoard format by default. Connect this with TensorBoard is a visualization program to view and inspect information on models and their data and we get ---- another way to visualize, visualize, visualize!


There are multiple ways you can check out TensorBoard: 

**VS Code - Notebooks / Python Scripts:** - Launching "Python: Launch TensorBoard" in Command Palette
**Jupyter / Collab Notebooks:** - Make sure TensorBoard is installed, load it with *%load_ext tensorboard* and then view the results with *%tesnorboard --logdir DIR_WITH_LOGS*.

What's great about TensorBoard is that you can upload the experiment results to *tensorboard.dev* to share them publicly with other people.

Try running the following code in Google Colab / Jupyter Notebook to start an interactive TensorBoard session to view TensorBoard files in the runs/ directory

* *%load_ext tensorboard* - this loads TensorBoard
* *%tensorboard --logdir runs* - this indicates that TensorBoard will use the *runs/* directory


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs --host=127.0.0.1 --port=6006

**Create Helper Function For Building *SummaryWriter()* Instances**

We don't necessarily need to put the *SummaryWriter()* constructor in the *train()* function. We can make this more modular by putting it into a seperate function and just calling it in *train()*. 

So, let's make a helper function that can takes in an argument that specifies the name of the directory that it is going to create to store the results of an experiment. Basically, since we want to run multiple experiments, each has to have their own directory for their results and that's the idea of this helper function.

We'll be tracking the following things:

1. **Experiemnt Date & Timestamp** - When was the experiment conducted ?
2. **Experiment Name** - What is the name for the experiment?
3. **Model Name** - What was the name of the model used?
4. **Extra** - Anything else to add?

Just like working with manual transforms, you can have plenty of flexibility with what you want to track. That's why there's an *extra* section for you to have the option of tracking more info. 

Create a helper function called *create_writer()* which produces a *SummaryWriter()* instance which sends the output to a custom *log_dir*.

The main idea is that the directories would be assembled into something akin to this:

*runs/YYYY-MM-DD/experiment_name/model_name/extra*

In [ ]:
def create_writer(experiment_name: str,
                 model_name: str,
                 extra: str=None) -> torch.utils.tensorboard.writer.SummaryWriter():

    # Get timestamp of the current date
    timestamp = datetime.now().strftime("%Y-%m-%d")
    if extra:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", timestamp, experiment_name, model_name)

    print(f"[INFO] Created SummaryWriter, Saving To: {log_dir}...")
    return SummaryWriter(log_dir=log_dir)

Let's try out our new function!

In [ ]:
example_writer = create_writer(experiment_name="data_10_percent",
                              model_name="efficientnet_b0",
                              extra="10_epochs")

**Update Train() Function To Include *Writer* Parameter**

Since our *create_writer* function is working fine, let's integrate that with the *train()* function. We'll give the *train()* function the ability to create a seperate writer instance each time it is called. What this does is that if we're running different experiments, calling *train()* multiple times to run each experiment, would mean that it will create a unique writer that logs the results for that experiment alone. 

Basically -> one experiment = one writer = one log directory per experiment

The changes that we'll be making is to add a *writer* parameter to *train()* and then also check if there is an existing *writer*, if there is then we'll track the information there on that already existing one, else we'll make a new one. That way we don't create unecessary *writer* instances.


In [ ]:
def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    epochs: int,
    device: torch.device,
    writer: torch.utils.tensorboard.writer.SummaryWriter
) -> Dict[str, List]:

    results = {
              "train_loss": [],
              "train_acc": [],
              "test_loss": [],
              "test_acc": []
    }

    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device
        )

        test_loss, test_acc = test_step(
            model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn,
            device=device
        )

        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

        print(f"Epochs: {epoch+1} | "
               f"Train Loss: {train_loss:.4f} | "
               f"Train Accuracy: {train_acc:.4f} | "
               f"Test Loss: {test_loss:.4f} | "
               f"Test Accuracy: {test_acc:.4f}")

        if writer:
            writer.add_scalars(main_tag="Loss",
                               tag_scalar_dict={"train_loss": train_loss,
                                              "test_loss": test_loss},
                               global_step=epoch
            )

            writer.add_scalars(main_tag="Accuracy",
                               tag_scalar_dict={"train_acc": train_acc,
                                              "test_acc": test_acc},
                               global_step=epoch
            )

            writer.close()
        else:
            pass

    return results

**Running Multiple Modelling Experiments**

Now that we have TensorBoard ready to use with our *train()* function, we're technically ready to start a series of modelling experiments. Before, we've been running one experiment and then inspect the outputs one by one. Let's see if we can run multiple experiments and look at their results in one go.

**Which Experiments To Run?**

This is always the big ol question that many ponder around when deciding what to do with machine learning. 

That's because there's literally a lot of things that you can switch around. The freedom of having to choose what you want is both exciting and maddening. Imagine being in a grocery store and you want a can of soda but there's 100 different brands to choose from. That's overload!

Luckily, in our case we can just experiment, experiment, experiment. Try as many things as you want as long as you're not constrained by the time and resource that is available to you. 

Just to glance at a few things that you can experiment around with:

* Number of epochs
* Number of hidden Units/layers
* Amount of data
* Learning rate
* Application of data augmentations
* Changing model architectures

Each of these hyperparameters also involve many options to choose from but with consistent experimentation you'll eventually get an *intuition* of the specific things that might have considerable effect on your model. 

Of course, there's always no guarantee. It's always part science and magic. 

But there is a recipe for success. That range of 'success' is quite up to debate but there is one guarantee there is - the bigger your model means more learnable parameters and the more data that you have means more opportunities to learn. Mix these two together and you will get better performance. 

Since we're still starting out with machine learning, it's much better to start from small and scale it up from there. 

Ideally we want our experiments to run from a few seconds to a few minutes. It shouldn't take longer than that. 

The faster we finish our experiments means that we get more opportunity to see what doesn't work and then do the things that does work. 

**What Experiments To Run Now?**

We want to improve our model powering FoodVision Mini without it getting too big. The ideal model we want would have a test accuracy of 90%+ and also doesn't take too long to train and perform inference on. 

Since we have so many options, let's just simplify it to these three combinations:

1. Increasing the amount of data from 10% to 20%.
2. Trying a different model and comparing the two.
3. Testing different training times - 5 epochs vs 10 epochs.

All in all that's 8 different experiments to run. 

We start with:

1. 10% Data - efficientnet_b0 - 5
2. 10% Data - efficientnet_b2 - 5
3. 10% Data - efficientnet_b0 - 10
4. 10% Data - efficientnet_b2 - 10
5. 20% Data - efficientnet_b0 - 5
6. 20% Data - efficientnet_b2 - 5
7. 20% Data - efficientnet_b0 - 10
8. 20% Data - efficientnet_b2 - 10

Notice the scaling. We started with an experiment with the floor parameters with 10% data, a lower version of efficientnet and only 5 epochs while our last experiment has the ceiling parameters with 20% data, a higher version of efficient net and 10 epochs.

We slowly increase the datasize, model version, and number of epochs. At the end of the series, the last experiment would have essentially doubled all its parameters from the start experiment. 

**Note**: There truly is no limit to the number of experiments you can take but we have to start somewhere. It's important to note that we're only touching a very small amount of options that we have. Not to mention that we're only using 10%-20% of the original data in Food101 Dataset. If the model is working well then maybe we can expand that further and consider other classes in Food101. 

**Downloading Different Datasets**

Since we're trying to run different experiments, there's one thing to prep before we start and that's to make sure that we have the datasets we need to work with. We need to make two seperate datasets that indicate one is 10% of the data and the other 20%. While we did make one earlier, we'll redo that again just to make everything a bit clearer.

Both will use the same testing dataset for consistency. Specifically, we're just goint to use the 10% testing set. 

Let's get started.

In [ ]:
data_10_percent_path = data_setup.get_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip", 
                                           target="data_10_percent")

data_20_percent_path = data_setup.get_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip", 
                                           target="data_20_percent")

**Setting Up Train/Test Paths**

In [ ]:
train_dir_10_percent = data_10_percent_path/'train' 
train_dir_20_percent = data_20_percent_path/'train'

test_dataset = data_10_percent_path/'test'

print(f"Training Directory 10% - {train_dir_10_percent}")
print(f"Training Directory 20% - {train_dir_20_percent}")
print(f"Testing Directory 10% - {test_dataset}")

**Transform Datasets & Create DataLoaders**

In [ ]:
simple_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                                 std=[0.229,0.224,0.225]),
])

simple_transforms

In [ ]:
BATCH_SIZE = 32

train_10_dataloader, test_dataloader, class_list = data_setup.create_dataloaders(
    train_dir=train_dir_10_percent,
    test_dir=test_dataset,
    transforms=simple_transforms,
    batch_size=BATCH_SIZE,
    num_workers=0  
)

train_20_dataloader, test_dataloader, class_list = data_setup.create_dataloaders(
    train_dir=train_dir_20_percent,
    test_dir=test_dataset,
    transforms=simple_transforms,
    batch_size=BATCH_SIZE,
    num_workers=0
)

print(f"Number of batches of size {BATCH_SIZE} in 10 percent training: {len(train_10_dataloader)}")
print(f"Number of batches of size {BATCH_SIZE} in 20 percent training: {len(train_20_dataloader)}")
print(f"Number of batches of size {BATCH_SIZE} in testing dataloader: {len(test_dataloader)}")
print(f"Number of classes: {len(class_list)} | Class names: {class_list}")

**Build Feature Extractor Models**

Time to build the models. We're going to build two obviously:

1. *torchvision.models.efficientnet_b0()* + pretrained backbone + customized classifier head
2. *torchvision.models.efficientnet_b2()* + pretrained backbone + customized classifier head

There's nothing new in this segment. We'll basically freeze the base layers and then update the classifier head and add in the change for our outputs to fit the number of classes that we have. 

Remember! While these two models are similiar in the sense that they're both the same architecture, they're still not exactly the same because *effnetb2* has a different number of layers and parameters. 

The first thing that you should do when working with new models is to always check how they're built. Always inspect the input and output shapes to make sure that you have the appropriate data transforms to fit in the model. 

Let's create the models and their associated weights then take a look at their summaries from *torchinfo.summary()*

In [ ]:
effnetb0_weights = models.EfficientNet_B0_Weights.DEFAULT
effnetb0_model = models.efficientnet_b0(weights=effnetb0_weights)

effnetb2_weights = models.EfficientNet_B2_Weights.DEFAULT
effnetb2_model = models.efficientnet_b2(weights=effnetb2_weights)

In [ ]:
summary(model=effnetb0_model,
        input_size=(32,3,224,224), # ensure that this is input_size and not input_shape
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
summary(model=effnetb2_model,
        input_size=(32,3,224,224), # ensure that this is input_size and not input_shape
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Notice how *effnetb2* has almost double the amount of trainable parameters compared to *effnetb0*. The input and output shapes of some layers inside have also changed.

Remember how we setup the classifier head and freezed the base layer before? We had to manually do that. That'll be very tedious once we start working with multiple experiments. So this time, we'll create a helper function to do all that so we can just call it instead of having to repeat the process again and again.

In [ ]:
# We have the same number of out features so we'll set it as a global variable

OUT_FEATURES = len(class_list)

def create_effnetb0():

    weights = models.EfficientNet_B0_Weights.DEFAULT
    model = models.efficientnet_b0(weights=effnetb0_weights)
    
    for param in model.features.parameters():
        param.requires_grad = False

    set_seeds()

    model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(1280, OUT_FEATURES, bias=True)
    )

    model.name = "effnetb0"
    print(f"[INFO] Created Model - {model.name}")
    return model

def create_effnetb2():
    
    weights = models.EfficientNet_B2_Weights.DEFAULT
    model = models.efficientnet_b2(weights=effnetb2_weights)

    for param in model.features.parameters():
        param.requires_grad = False

    set_seeds()

    model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(1408, OUT_FEATURES, bias=True)
    )

    model.name = "effnetb2"
    print(f"[INFO] Created Model - {model.name}")
    return model


Looking good! We can now make multiple models pretty easily with these helper functions. Let's try calling them!

In [ ]:
effnetb0 = create_effnetb0().to(device)
effnetb2 = create_effnetb2().to(device)

In [ ]:
summary(model=effnetb0,
        input_size=(32,3,224,224), # ensure that this is input_size and not input_shape
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
summary(model=effnetb2,
        input_size=(32,3,224,224), # ensure that this is input_size and not input_shape
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Example the following values and compare for both *effnetb0* and *effnetb2* - Total Parameters | Trainable Parametes. Make sure to also examine the models before they were frozen and after. 

Overall, we can see that the total parameters for both are almost double. Meaning that *effnetb2* has double the learning opportunities. But at the end of the day, the classifier heads are pretty similiar. Will these extra parameters make a difference in our loss and accuracy results?

The only way to find out is to run both!

**Create Experiments & Setting Up Training Code**

We'll need to format these things into lists and dicts if we want to work with them on a series so let's get started with the following:

1. Create a list of the number of epochs that we want to use *[5,10]*.
2. Create a list of the models that we want to run experiments on *["effnetb0", "effnetb2"]*.
3. Create a dictionary for the different training DataLoaders.

In [ ]:
num_epochs = [5,10]
model_list = ["effnetb0", "effnetb2"]
train_dataloaders = {"data_10_percent": train_10_dataloader,
                     "data_20_percent": train_20_dataloader}

Now that we have created our lists and dictionary for the models and their associated epochs, we can start writing the code that iterates through each of the options and work with the different combinations. 

In addition, we will also save the model at the end of each experiment so we can load the best model and make predictions with it.

Let's run through the steps in making this iterating function - 

1. Set the random seeds so that we can have reproducable experiments.
2. Keep track of the experiment numbers for print outs.
3. Loop through the *train_dataloaders* dictionary items for each of the different training DataLoaders.
4. Loop through the epochs.
5. Loop through the different models.
6. Print out the current experiment configuration so we can know where we are.
7. Check the target model and create an instance of it - either *effnetb0* or *effnetb2*.
8. Choose the loss function and optimizer for the experiment.
9. Train the model with *train* and pass the required details to the *writer* parameter.
10. Save the trained model with the chosen file name to the directory specified with *save_model()* from *utils.py*

We will also use *%%time* so that we can track how long all of the experiments took to run.

In [ ]:
train_dataloaders

In [ ]:
train_dataloaders.items()

In [ ]:
%%time
set_seeds()

experiment_track = 0
# We need to set the train_dataloaders.item() because we want to iterate through the key-value pairs and assign them to 
# individual variables, in this case, dataloader_name for key and train_dataloader for value.
for dataloader_name, train_dataloader in train_dataloaders.items():
    for epochs in num_epochs:
        for model_name in model_list:
            experiment_track += 1 
            print(f"[INFO] - EXPERIMENT #{experiment_track}")
            print(f"[INFO] Experiment Configuration - DataLoader: {dataloader_name} | Epochs: {epochs} | Model: {model_name}")
            if model_name == "effnetb0":
                model_instance = create_effnetb0().to(device)
            else:
                model_instance = create_effnetb2().to(device)

            loss_fn=torch.nn.CrossEntropyLoss()
            optimizer=torch.optim.Adam(model_instance.parameters(), lr=0.001)

            results = train(
                model=model_instance,
                train_dataloader=train_dataloader,
                test_dataloader=test_dataloader,
                loss_fn=loss_fn,
                optimizer=optimizer,
                epochs=epochs,
                device=device,
                writer=create_writer(experiment_name=dataloader_name, 
                                     model_name=model_name, 
                                     extra=f"{epochs}_epochs")
            )
            save_filepath = f"08_{model_name}_{dataloader_name}_{epochs}_epochs.pth"
            utils.save_model(model=model_instance,
                      target_dir="models",
                      model_name=save_filepath)
            print("-"*50 + "\n")

**Inspecting Results In TensorBoard**

Ain't that exciting? We just trained eight models with different hyperparameters in one single sweep. Let's take a look at the results for each one in TensorBoard.

![Display](images/08-tensorboard-series-experiment-results.png)

Looking at the results, we can see that the models which achieved the lowest loss rate is *effnetb0* with 10 epochs. There doesn't seem to be a difference in the dataset because both 10% and 20% are tied for 1st place. 

**Loading Best Model + Creating Predictions**

Seems like we have two options to choose from with our best model. Let's take the one that has the following configuration:

* *Effnetb2* which has double the parameters
* 20% training data
* and 10 epochs

Basically, the model that used much more data and computation was the one that performed the best. That's not to say that the other models didn't perform good. They all gave out some promising results. One even achieved a pretty good result at half the time it took for our best performing model to finish. 

From the perspective of the results, it seems like the a major factor is the number of epochs involved in training. You could always run more experiments to test these results and see more about the key items influencing the results. 

For now, let's just import the best performing model that was saved to the models folder. It's time to *visualize, visualize, visualize*

As always, we can just create a new instance of that certain model and just load the *state_dict* to that new instance. 

In [ ]:
best_model_path = "models/08_effnetb2_data_20_percent_10_epochs.pth"
best_model_path

In [ ]:
best_model = create_effnetb2()

In [ ]:
best_model.load_state_dict(torch.load(best_model_path))

We took the weights from the path in our models, instantiated a new effnet model and then just loaded the weights into it. This method is the best practice to take when loading models. But there's something we have to do first before we start predicting on this model.

We have to first check the filesize. Why is that important? It's because we have to consider whether or not the model will load quickly. If it's too large then it will be hard to deploy. That is why we have to make sure that filesize is appropriate.

In [ ]:
effnetb2_model_size = Path(best_model_path).stat().st_size // (1024*1024)
# .stat() grabs multiple details (including file size) from the file and returns an object and then we access .st_size
# which is the file size in bytes. We convert this into MB by floor dividing with (1024*1024)
print(f"EfficientNetB2 Feature Extractor Model Size: {effnetb2_model_size} MB")

So the best size for our model is 29 MB. Is that a good number? We'll see later once we deploy it.

Let's make some predictions with it. We import the *predict.py* from *Going Modular* and then use the *predict_image* function. We'll use images from the 20% pizza, steak, sushi testing dataset. We will randomly select a subset of these filepaths and then pass this to the predict function.

In [ ]:
test_dir

In [ ]:
random.seed(42)

num_images_plot = 3
test_image_path_list = list(test_dir.glob("*/*.jpg"))
test_rand_sample = random.sample(test_image_path_list, 3)

for image in test_rand_sample:
    print(image)
    predict.predict_image(
        model=best_model,
        image_path=image,
        transform=simple_transforms,
        class_list=class_list,
        device=device
    )

In [ ]:
test_preds = []
best_model.eval()
with torch.inference_mode():
    for X,y in tqdm(test_dataloader):
        X,y = X.to(device), y.to(device)
        pred = best_model(X)
        pred_probs = torch.softmax(pred, dim=1)
        pred_labels = torch.argmax(pred_probs, dim=1)
        test_preds.append(pred_labels)

# We do a list comprehension on test_preds to transfer the device to cpu because if we keep it on cuda then it'll be hard to look at the 
# formation of the values because there would be device=''cuda:0' section
test_preds = [tensor.cpu() for tensor in test_preds]
print(f"Before Cat: {test_preds}\n")
# We apply concat because the tensors are seperated into batches and we want all of the values in one dimension.
test_preds = torch.cat(test_preds).cpu()
print(f"After Cat: {test_preds}")

In [ ]:
test_truth = torch.cat([y for X, y in test_dataloader])
test_truth

In [ ]:
confmat = ConfusionMatrix(num_classes=len(class_list), task='multiclass')
confmat_tensor = confmat(preds=test_preds,
                        target=test_truth)

fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(),
    class_names=class_list,
    figsize=(10,7)
)

Notice how our model now has a high confidence on its predictions compared to our previous model. In addition, if we check our confusion matrix, it's coming out very accurate with all it's predictions with just a very small margin of error. 

That's fine because model's aren't perfect. 

One thing is for sure is that we were able to get the best model now because it is very sure with it's correct prediction. 

Let's try predicting with a custom image.

**Predicting Custom Image With Best Model**

Let's try using the same old test custom image before and just compare that. Use the same function that we used before and experiment to see what the results are.

In [ ]:
custom_image_path = Path('custom/image-3.jpg')

predict.predict_image(
    model=best_model,
    image_path=custom_image_path,
    transform=simple_transforms,
    class_list=class_list,
    device=device
)

Seems like this model performs really well when it comes to identifying pizza but it depends. It can be beaten by *effnet0* when working with steaks on some certain images. There isn't really a guaranteed result as to which model is better but if that's the question then you can simply just take a look at the metrics and generally, the one with the lowest loss and the highest accuracy is better.

**Main Takeaways**

We've successfully completed the major parts of how to work with PyTorch. We now know how to work with Datasets, DataLoaders, and how to make data ready for training/testing. We can pick a pretrained model and then use it. We also created different helper functions to make the entire process easier. Not to mention that we managed to create an improved version of FoodVision Mini alongside creating a series of experiments and tracking/viewing their results in one go.

Let's summarize the key points again:

1. Always *"experiment, experiment, experiment"*. Experiments always bring experience and you could always use more of that.
2. Make sure that experiments at the start should be kept at a minimum for the duration that they run. Start small and scale up.
3. The faster you get your experiments done, the faster you'll be able to identify how to work the problem.
4. Once you found a probable solution, scale up the process. See how it works on a bigger picture.
5. Taking the time to build an environment for experimenting might take some time but the benefits of a faster/flexible work process will  make it worth it at the end. 

**Exercises**